In [1]:
# define the model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Add, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import optimizers

img_feature_a = Input(shape=(1024*4,))
img_feature_b = Input(shape=(1024*4,))

h_1 = Dense(256, activation='relu')
bn_1 = BatchNormalization()
do_1 = Dropout(0.5)
#h_2 = Dense(64, activation='relu')
#bn_2 = BatchNormalization()
#do_2 = Dropout(0.5)
s = Dense(1)

def aesthetic_layer(x):
    x = h_1(x)
    x = bn_1(x)
    x = do_1(x)
    #x = h_2(x)
    #x = bn_2(x)
    #x = do_2(x)
    return s(x)

score_a = aesthetic_layer(img_feature_a)
score_b = aesthetic_layer(img_feature_b)

negated_score_b = Lambda(lambda x: -x, output_shape=(1,))(score_b)
diff = Add()([score_a, negated_score_b])

output = Activation("sigmoid")(diff)

ranknet = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

#optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ranknet.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

ranknet.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 4096)          0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 4096)          0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 256)           1048832     input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 256)           1024        dense_1[0][0]           

In [66]:
# define the data
from keras.preprocessing.image import *
import os
import numpy as np
import lmdb
import random

env = lmdb.open("./data/mobilenet_feature/")
txn = env.begin()

def get_feature(imgid):
    str_feature = txn.get(imgid)
    return np.fromstring(str_feature, np.float32)

# 自定义DirectoryIterator类，可以返回自定义的label
class CustomDirectoryIterator(DirectoryIterator):  
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x1 = np.zeros((current_batch_size,) + self.target_size, dtype=K.floatx())
        batch_x2 = np.zeros((current_batch_size,) + self.target_size, dtype=K.floatx())
        batch_y = np.zeros((current_batch_size, ) , dtype=K.floatx())
        # build batch of image data
        for i, j in enumerate(index_array):
            fname =  self.filenames[j]
            fname = os.path.basename(fname)
            fname, _ = os.path.splitext(fname)
            imgA, imgB, cmpret = fname.split("_")
            x1 = get_feature(imgA)
            x2 = get_feature(imgB)
            batch_x1[i] = x1
            batch_x2[i] = x2
            batch_y[i] = int(cmpret)
        return [batch_x1, batch_x2], batch_y

# 定义批处理的数据集大小：较小的batch_size可以增加权重调整的次数，同时节省内存的开销
batch_size = 64 

# 图片预处理工具类
train_IDG = ImageDataGenerator(vertical_flip=True, zoom_range=0.1)
valid_IDG = ImageDataGenerator()

# 从目录文件中流式读取数据，避免训练中一次性加载爆内存
train_batch = CustomDirectoryIterator("./data/train/", train_IDG, 
                                      target_size=(1024*4, ), batch_size=batch_size, shuffle=True)
valid_batch = CustomDirectoryIterator("./data/valid/", valid_IDG, 
                                      target_size=(1024*4, ), batch_size=batch_size, shuffle=True)

Found 375764 images belonging to 1 classes.
Found 4277 images belonging to 1 classes.


In [68]:
ranknet.fit_generator(train_batch, steps_per_epoch=train_batch.samples // batch_size, epochs=3,
                       validation_data=valid_batch, validation_steps=valid_batch.samples // batch_size)

Epoch 1/3
5871/5871 [==============================] - 13s - loss: 0.5326 - acc: 0.7458 - val_loss: 0.6808 - val_acc: 0.5953
Epoch 2/3
5871/5871 [==============================] - 13s - loss: 0.5280 - acc: 0.7452 - val_loss: 0.6804 - val_acc: 0.6000
Epoch 3/3
5871/5871 [==============================] - 13s - loss: 0.5245 - acc: 0.7444 - val_loss: 0.6799 - val_acc: 0.6024


In [ ]:
model.save_weights("./data/ranknet/ranknet.h5")